<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#决策树" data-toc-modified-id="决策树-1">决策树</a></span></li><li><span><a href="#1.-特征选择" data-toc-modified-id="1.-特征选择-2">1. 特征选择</a></span><ul class="toc-item"><li><span><a href="#1.1-熵" data-toc-modified-id="1.1-熵-2.1">1.1 熵</a></span></li><li><span><a href="#1.2-条件熵" data-toc-modified-id="1.2-条件熵-2.2">1.2 条件熵</a></span></li><li><span><a href="#1.3-信息增益" data-toc-modified-id="1.3-信息增益-2.3">1.3 信息增益</a></span></li><li><span><a href="#1.4-信息增益率" data-toc-modified-id="1.4-信息增益率-2.4">1.4 信息增益率</a></span></li></ul></li><li><span><a href="#2.-决策树生成" data-toc-modified-id="2.-决策树生成-3">2. 决策树生成</a></span><ul class="toc-item"><li><span><a href="#算法1-信息增益及信息增益率的算法" data-toc-modified-id="算法1-信息增益及信息增益率的算法-3.1">算法1 信息增益及信息增益率的算法</a></span></li><li><span><a href="#2.1-ID3-算法" data-toc-modified-id="2.1-ID3-算法-3.2">2.1 ID3 算法</a></span></li><li><span><a href="#2.2-C4.5-算法" data-toc-modified-id="2.2-C4.5-算法-3.3">2.2 C4.5 算法</a></span></li></ul></li><li><span><a href="#3.-决策树剪枝" data-toc-modified-id="3.-决策树剪枝-4">3. 决策树剪枝</a></span><ul class="toc-item"><li><span><a href="#3.1-预剪枝" data-toc-modified-id="3.1-预剪枝-4.1">3.1 预剪枝</a></span></li><li><span><a href="#3.2-后剪枝" data-toc-modified-id="3.2-后剪枝-4.2">3.2 后剪枝</a></span><ul class="toc-item"><li><span><a href="#算法2-树的剪枝算法" data-toc-modified-id="算法2-树的剪枝算法-4.2.1">算法2 树的剪枝算法</a></span></li></ul></li></ul></li><li><span><a href="#参考文献" data-toc-modified-id="参考文献-5">参考文献</a></span></li></ul></div>

相关文章：

[机器学习 | 目录](https://blog.csdn.net/weixin_45488228/article/details/99691709)

[监督学习 | ID3 决策树原理及Python实现](https://blog.csdn.net/weixin_45488228/article/details/98665115)

[监督学习 | 决策树之Sklearn实现](https://blog.csdn.net/weixin_45488228/article/details/98751821)

[监督学习 | 决策树之网络搜索](https://blog.csdn.net/weixin_45488228/article/details/99253498)

本文大部分内容搬运自李航老师的《统计学习方法》<sup>[1]</sup>，以给出决策树算法较为完整的定义，关于 ID3 算法的更多推到过程以及例子、Python实现，可以参考这篇[文章](https://blog.csdn.net/weixin_45488228/article/details/98665115)，关于决策树算法的 Sklearn 实现，可以参考这篇[文章](https://blog.csdn.net/weixin_45488228/article/details/98751821)。

# 决策树

决策树是一种基本的分类与回归方法。在分类问题中，表示基于特征对实例进行分类的过程。它可以认为是 if-then 规则的集合，也可以认为是定义在特征空间与类空间上的条件概率分布。

决策树学习通常包括 3 个步骤：`特征选择`、`决策树的生成`和`决策树的修剪`。这些决策树学习的思想主要来源于由 Quinlan 在 1986 年提出的 ID3 算法和 1993 年提出的 C4.5 算法，以及由 Breiman 等人在 1984 年提出的 CART 算法。这三个算法最大的不同在于特征选择的指标上，三者分别使用：信息增益、信息增益率以及基尼指数作为特征选择指标。本文将介绍 ID3 以及 C4.5 算法，并在下一篇文章中介绍 CART 算法。

# 1. 特征选择

特征选择在于选取对训练数据具有分类能力的特征，这样可以提高决策树学习的效率。如果利用一个特征进行分类的结果与随机分类的结果没有很大差别，则称这个特征是没有分类能力的。

通常特征选择的准则是信息增益或信息增益率。

## 1.1 熵

`熵`（entropy）是表示随机变量不确定性的度量。设 $X$ 是一个取有限个值的离散随机变量，其概率分布为：

$$P(X=x_i)=p_i, \quad i=1,2,...,n$$

则随机变量 $X$ 的熵定义为：

$$H(X)=-\sum_{i=1}^np_i\log p_i \tag{1}$$

若 $p_i=0$，则定义 $0*\log0=0$，通常式 (1) 中的对数以 2 或 e 为底，单位为比特（bit）或纳特（nat）。由此可知，熵只依赖于 $X$ 的分布，而与 $X$ 的取值无关，所以也可以将 $X$ 的熵记做 $H(p)$，即：

$$H(p)=-\sum_{i=1}^np_i\log p_i \tag{2}$$

熵越大，随机变量的不确定性就越大，由此可以验证：

$$0\leq H(p)\leq \log n \tag{3}$$

## 1.2 条件熵

设由随机变量 $(X,Y)$，其联合概率分布为：

$$P(X=x_i,Y=y_j)=p_{ij},\quad i=1,2,...,n;j=1,2,...,m$$

`条件熵` $H(Y|X)$ 表示在已知随机变量 $X$ 的条件下随机变量 $Y$ 的不确定性，其定义为 $X$ 给定条件下 $Y$ 的条件概率分布的熵的 $X$ 的数学期望：

$$H(Y|X)=\sum_{i=1}^n p_iH(Y|X=x_i)\tag{4}$$

其中 $p_i=P(X=x_i), \quad i=1,2,...,n$。

当熵和条件熵中的概率由数据估计得到时，所对应的熵与条件熵分别称为`经验熵`（emprical entropy）和`经验条件熵`（empirical conditional emtropy）。

## 1.3 信息增益

`信息增益`（information gain）表示得知特征 $X$ 的信息而使类 $Y$ 的信息不确定性减少的程度。

特征 $A$ 对训练数据集 $D$ 的信息增益 $g(D,A)$，定义为集合 $D$ 的经验熵 $H(D)$ 与特征 $A$ 给定条件下 $D$ 的经验条件熵 $H(D|A)$ 之差，即：

$$g(D,A)=H(D)-H(D|A) \tag{5}$$

一般地，熵 $H(Y)$ 与条件熵 $H(D|A)$ 之差称为 `互信息`（mutual information）。决策树学习中的信息增益等价于训练数据集中类与特征的互信息。

## 1.4 信息增益率

信息增益率的大小时相对于训练数据集而言的，在分类问题困难时，也就是说在训练数据集的经验熵大的时候，信息增益值会偏大，信息增益值会偏大。反之，信息增益值会偏小。使用`信息增益比`（information gain ratio）可以对这一问题进行校正，这是特征选择的另一准则。

特征 $A$ 对训练数据集 $D$ 的`信息增益率` $g_R(D,A)$ 定义为其信息增益 $g(D,A)$ 与训练数据集 $D$ 的经验熵 $H(D)$ 之比：

$$g_R(D,A)=\frac{g(D,A)}{H(D)} \tag{6}$$

# 2. 决策树生成

***

设训练集数据为 $D$，$|D|$ 表示其样本容量，即样本个数。

设有 $K$ 个类 $C_k ,k=1,2,...,K$，$|C_k|$ 为属于类 $C_k$ 的样本个数，因此 $\sum_{k=1}^K|C_k|=|D|$。

设特征 $A$ 有 $n$ 个不同的取值 $\{a_1,a_2,...,a_n\}$，根据特征 $A$ 的取值将 $D$ 划分为 $n$ 个子集 $D_1,D_2,...,D_n$，$|D_i|$ 为 $D_i$ 的样本个数，$\sum_{i=1}^n|D_i|=D$。

记子集 $D_i$ 中属于类 $C_k$ 的样本的集合为 $D_{ik}$，即 $D_{ik}=D \bigcap C_k$，$|D_{ik}|$ 为 $D_{ik}$ 的样本个数。

因此，信息增益以及信息增益率计算如下：

## 算法1 信息增益及信息增益率的算法

**输入**：训练数据集 $D$ 和特征 $A$；

**输出**：特征 $A$ 对训练数据集 $D$ 的信息增益 $g(D,A)$ 以及信息增益率 $g_R(D,A)$。

（1）计算数据集 $D$ 的经验熵 $H(D)$：

$$H(D)=-\sum_{i=1}^n \frac{|C_k|}{|D|}log_2\frac{|D_k|}{|D|} \tag{7}$$

（2）计算特征 $A$ 对数据集 $D$ 的经验条件熵 $H(D|A)$：

$$
\begin{aligned}
H(D|A)& =\sum_{i=1}^n \frac{|D_i|}{|D|}H(D_i)\\
& = - \sum_{i=1}^n \frac{|D_i|}{|D|} \sum_{i=1}^n \frac{|D_{ik}|}{|D_i|}log_2\frac{|D_{ik}|}{|D_i|}\\
\end{aligned}\tag{8}
$$

（3）计算信息增益：

$$g(D,A)=H(D)-H(D|A) \tag{9}$$

（4）计算信息增益率：

$$g_R(D,A)=\frac{g(D,A)}{H(D)} \tag{10}$$

## 2.1 ID3 算法

**输入**：训练数据集 $D$，特征集 $A$，阈值 $\varepsilon$；

**输出**：决策树 $T$。

（1）若 $D$ 中所有实例属于同一类 $C_k$，则 $T$ 为单节点树，并将类 $C_k$ 作为该节点的类标记，返回 $T$；

（2）若 $A=\oslash$，则 $T$ 为单节点树，并将 $D$ 中实例数最大的类 $C_k$ 作为该节点的类标记，返回 $T$；

（3）否则，按算法1（1-3）计算 $A$ 中各特征对 $D$ 的信息增益，选择信息增益最大的特征 $A_g$；

（4）如果 $A_g$ 的信息增益小于阈值 $\varepsilon$，则置 $T$ 为单节点树，并将 $D$ 中实例数最大的类 $C_K$ 作为该节点的类标记，返回 $T$；

（5）否则，对 $A_g$ 的每一可能值 $a_i$，依 $A_g=a_i$ 将 $D$ 分隔为若干非空子集 $D_i$，将 $D_i$ 中实例数最大的类作为标记，构建子节点，由节点及其自己点构造数 $T$，返回 $T$；

（6）对第 $i$ 个子节点，以 $D_i$ 为训练集，以 $A-\{A_g\}$ 为特征集，递归地调用步(1)～步(5)，得到子树 $T_i$，返回 $T_i$。

## 2.2 C4.5 算法

C4.5 与 ID3 相比，只是将 ID3 算法中的信息增益换成了信息增益率，因此有：

**输入**：训练数据集 $D$，特征集 $A$，阈值 $\varepsilon$；

**输出**：决策树 $T$。

（1）若 $D$ 中所有实例属于同一类 $C_k$，则 $T$ 为单节点树，并将类 $C_k$ 作为该节点的类标记，返回 $T$；

（2）若 $A=\oslash$，则 $T$ 为单节点树，并将 $D$ 中实例数最大的类 $C_k$ 作为该节点的类标记，返回 $T$；

（3）否则，按算法1（1-4）计算 $A$ 中各特征对 $D$ 的信息增益，选择信息增益最大的特征 $A_g$；

（4）如果 $A_g$ 的信息增益小于阈值 $\varepsilon$，则置 $T$ 为单节点树，并将 $D$ 中实例数最大的类 $C_K$ 作为该节点的类标记，返回 $T$；

（5）否则，对 $A_g$ 的每一可能值 $a_i$，依 $A_g=a_i$ 将 $D$ 分隔为若干非空子集 $D_i$，将 $D_i$ 中实例数最大的类作为标记，构建子节点，由节点及其自己点构造数 $T$，返回 $T$；

（6）对第 $i$ 个子节点，以 $D_i$ 为训练集，以 $A-\{A_g\}$ 为特征集，递归地调用步(1)～步(5)，得到子树 $T_i$，返回 $T_i$。

# 3. 决策树剪枝

## 3.1 预剪枝

预剪枝是在决策树生成之前通过限制条件，来防止树过度生长而造成过拟合，常见的有：

1. 最大深度 max_depth

2. 每片叶子的最小样本数 min_samples_leaf

3. 每次分裂的最小样本数 min_samples_split

4. 最大特征数 max_features

关于这些参数的详细介绍，可以参考这篇[文章](https://blog.csdn.net/weixin_45488228/article/details/98751821)。

## 3.2 后剪枝

后剪枝先从训练集生成一颗完整决策树，通过向损失函数中增加模型复杂度惩罚来对已生成的决策树进行简化。

设树 $T$ 的叶节点个数为 $|T|$，$t$ 是树 $T$ 的叶节点，该叶节点有 $N_t$ 个样本点，其中 $k$ 类的样本点有 $N_{tk}$ 个，$k=1,2,...,K$，$H_t(T)$ 为叶节点 $t$ 上的经验熵，$\alpha \geq 0 $ 为参数，则决策树后剪枝的损失函数（Cost-Complexity Pruning）可以定义为：

$$C_{\alpha}(T)=\sum_{t=1}^{|T|}N_tH_t(T)+\alpha |T|\tag{11}$$

其中经验熵为：

$$H_t(T)=-\sum_{k} \frac{N_{ik}}{N_t}log\frac{N_{ik}}{N_t} \tag{12}$$

令 (11) 式右端第一项为 $C(T)$：

$$
\begin{aligned}
C(T)& =\sum_{t=1}^{|T|}N_tH_t(T) \\
& = -\sum_{t=1}^{|T|}N_t\sum_{k} \frac{N_{ik}}{N_t}log\frac{N_{ik}}{N_t} \\
\end{aligned}\tag{13}
$$

因此式 (11) 可以写作：

$$C_{\alpha}(T)=C(T)+\alpha |T| \tag{14}$$

其中，$C(T)$ 表示模型对训练数据的预测误差，即模型与训练数据的拟合程度；

$|T|$ 表示模型复杂度，参数 $\alpha \geq 0$ 控制两者之间的影响。较大的 $\alpha$ 促使选择较简单的模型，较小的 $\alpha$ 促使选择较复杂的模型， $\alpha=0$ 意味者只考虑模型与训练数据的拟合程度，不考虑模型的复杂度。

剪枝，就是当 $\alpha$ 确定时，选择损失函数最小的模型，即损失函数最小的子树。

当 $\alpha$ 确定时，子树越大，往往与训练数据的拟合越好，但是模型的复杂度越高；相反，子树越小，模型复杂度就越低，但是往往与训练数据的拟合不好，损失函数正好表示了对两者的平衡。

可以看到，ID3 和 C4.5 决策树生成只考虑了通过提高信息增益或信息增益率来对训练数据进行更好的拟合。而决策树剪枝通过优化损失函数还考虑了减小模型复杂度。

决策树生成学习局部的模型，而决策树剪枝学习整体的模型。

### 算法2 树的剪枝算法

**输入**：生成算法产生的整个树 $T$，参数 $\alpha$；

**输出**：修剪后的子树 $T_{\alpha}$。

（1）计算每个节点的经验熵；

（2）递归地从树的叶节点向上回缩：

$\quad \quad$设一组叶节点回缩到其父节点之前与之后的整体树分别为 $T_B$ 与 $T_A$，其对应的损失函数值分别是 $C_{\alpha}(T_B)$ 与 $C_{\alpha}(T_A)$，如果：

$$C_{\alpha}(T_A)\leq C_{\alpha}(T_B) \tag{15}$$

$\quad \quad$ 则进行剪枝，即将父节点变为新的叶节点。

（3）返回 (2) ，直到不能继续为止，得到损失函数最小的子树 $T_{\alpha}$。

# 参考文献

[1] 李航. 统计学习方法[M]. 北京: 清华大学出版社, 2012: 55-66.